In [1]:
# -*-coding:utf-8 -*-
import numpy as np
from bs4 import BeautifulSoup
import random

In [18]:
'''
函数功能：从页面读取数据，生成retX和retY列表
参数说明：
    retX：数据特征
    retY: 数据标签
    inFile：HTML文件
    yr：年份
    numPce：乐高部件数目
    origPrc：原价
返回：
    无
'''
def scrapePage(retX,retY,inFile,yr,numPce,origPrc):
    # 打开并读取HTML界面
    with open(inFile,encoding = 'utf-8') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    
    i=1
    # 根据HTML页面结构进行解析
    currentRow = soup.find_all('table',r="%d"%i)
    while(len(currentRow)!=0):
        currentRow = soup.find_all('table',r="%d"%i)
        title = currentRow[0].find_all('a')[1].text
        lwrTitle = title.lower()
        # 查找是否有全新标签
        if (lwrTitle.find('new') > -1) or (lwrTitle.find('nisb') > -1):
            newFlag = 1.0
        else:
            newFlag = 0.0
        
        # 查找是否已经标志出售，我们只收集已出售的数据
        soldUnicde = currentRow[0].find_all('td')[3].find_all('span')
        if len(soldUnicde) == 0:
            #print("商品 #%d 没有出售" % i)
            pass
        else:
            
            # 解析页面获取当前价格
            soldPrice = currentRow[0].find_all('td')[4]
            priceStr = soldPrice.text
            priceStr = priceStr.replace('$','')
            priceStr = priceStr.replace(',','')
            if len(soldPrice) >1:
                priceStr = priceStr.replace('Free shipping','')
            sellingPrice = float(priceStr)
            
            # 去掉不完整的套装价格
            if sellingPrice > origPrc * 0.5:
                #print("%d\t%d\t%d\t%f\t%f" % (yr, numPce, newFlag, origPrc, sellingPrice))
                retX.append([yr,numPce,newFlag,origPrc])
                retY.append(sellingPrice)
        i += 1
        
        # 找下一个套装信息
        currentRow = soup.find_all('table',r = '%d' %i)

In [6]:
def setDataCollect(retX,retY):
    scrapePage(retX,retY,'setHTML/lego8288.html',2006,800,49.99)
    scrapePage(retX,retY,'setHTML/lego10030.html',2002,3096,269.99)    
    scrapePage(retX,retY,'setHTML/lego10179.html',2007,5195,499.99)    
    scrapePage(retX,retY,'setHTML/lego10181.html',2007,3428,199.99)    
    scrapePage(retX,retY,'setHTML/lego10189.html',2008,5922,199.99)    
    scrapePage(retX,retY,'setHTML/lego10196.html',2009,3263,249.99)    

In [7]:
if __name__ == '__main__':
    lgX = []
    lgY = []
    setDataCollect(lgX,lgY)

D:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


2006	800	0	49.990000	85.000000
2006	800	0	49.990000	102.500000
2006	800	0	49.990000	77.000000
商品 #4 没有出售
2006	800	0	49.990000	162.500000
2002	3096	0	269.990000	699.990000
2002	3096	0	269.990000	602.000000
2002	3096	0	269.990000	515.000000
2002	3096	0	269.990000	510.000000
2002	3096	0	269.990000	375.000000
2002	3096	1	269.990000	1050.000000
2002	3096	0	269.990000	740.000000
2002	3096	1	269.990000	759.000000
2002	3096	0	269.990000	730.000000
2002	3096	1	269.990000	750.000000
商品 #11 没有出售
2007	5195	0	499.990000	910.000000
2007	5195	1	499.990000	1199.990000
2007	5195	0	499.990000	811.880000
商品 #4 没有出售
2007	5195	0	499.990000	1324.790000
2007	5195	1	499.990000	850.000000
2007	5195	1	499.990000	800.000000
2007	5195	0	499.990000	810.000000
2007	5195	1	499.990000	1075.000000
2007	5195	0	499.990000	1050.000000
2007	5195	1	499.990000	1199.990000
2007	5195	0	499.990000	1342.310000
2007	5195	1	499.990000	1000.000000
2007	5195	0	499.990000	1780.000000
2007	5195	0	499.990000	750.000000
商品 #16 没有出售
200

我们对没有的商品做了处理。这些特征分别为：出品年份、部件数目、是否为全新、原价、售价（二手交易）。

## 2、建立模型

我们已经处理好了数据集，接下来就是训练模型。首先我们需要添加全为0的特征X0列。
因为线性回归的第一列特征要求都是1.0。然后使用最简单的普通线性回归i，编写代码如下：

In [8]:
'''
函数说明：数据标准化
参数说明：
    xMat：数据集的特征
    yMat：数据集的标签
    
返回：
    标准化后的数据集
'''

def regularize(xMat,yMat):
    inxMat = xMat.copy() #数据拷贝
    inyMat = yMat.copy()
    yMean = np.Mean(yMat,0) # 压缩行，对各列求均值，返回 1* n 矩阵
    inyMat = yMat - yMean  # 据减去均值
    inMeans = np.mean(inxMat,0)
    inVar = np.Var(inxMat,0) # 求方差
    inxMat = (inxMat - inMeans) / inVar #数据减去均值除以方差实现标准化
    
    return inxMat,inyMat

In [9]:
'''
函数说明：计算平方误差和
参数说明：
    yArr：预测值
    yHatArr：真实值
返回：
    预测值与真实值的误差平方和
'''
def rssError(yArr,yHatArr):
    return ((yArr - yHatArr)**2).sum()

In [10]:
'''
函数说明：使用线性回归方法求回归系数
参数说明：
    xArr：数据集特征
    yArr：数据集标签
返回：
    ws：特征的回归系数
'''
def standRegress(xArr,yArr):
    xMat = np.mat(xArr)
    yMat = np.mat(yArr).T
    xTx = xMat.T * xMat   #根据推导的公示计算回归系数
    if np.linalg.det(xTx) == 0.0:
        print("矩阵为奇异矩阵,不能转置")
        return
    ws = xTx.I * (xMat.T*yMat)
    return ws

In [22]:
'''
函数说明：使用简单的线性回归
参数说明：
    无
'''
def useStandRegress():
    lgX = []
    lgY = []
    setDataCollect(lgX,lgY)
    
    data_num , features_num = np.shape(lgX)
    
    lgX1 = np.mat(np.ones((data_num,features_num + 1)))
    
    lgX1[:,1:5] = np.mat(lgX)
    ws = standRegress(lgX1,lgY)
    print('%f%+f*年份%+f*部件数量%+f*是否为全新%+f*原价' % (ws[0],ws[1],ws[2],ws[3],ws[4]))    

In [23]:
useStandRegress()

D:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


62846.112549-31.383276*年份+0.051156*部件数量-10.631107*是否为全新+1.918861*原价


In [24]:
'''
函数功能：岭回归求回归系数
参数说明：
    xMat：数据集特征
    yMat：数据集标签
    lam：缩减系数
返回：
    ws：回归系数
'''
def ridgeRegres(xMat, yMat, lam = 0.2):
    xTx = xMat.T * xMat
    denom = xTx + np.eye(np.shape(xMat)[1]) * lam
    if np.linalg.det(denom) == 0.0:
        print("矩阵为奇异矩阵,不能转置")
        return
    ws = denom.I * (xMat.T * yMat)
    return ws

In [25]:
'''
函数功能：岭回归预测
参数说明：
    xMat：数据集特征
    yMat：数据集标签
    lam：缩减系数
返回：
    ws：回归系数矩阵
'''
def ridgeTest(xArr, yArr):
    xMat = np.mat(xArr); yMat = np.mat(yArr).T
    #数据标准化
    yMean = np.mean(yMat, axis = 0)                        #求均值
    yMat = yMat - yMean                                    #数据减去均值
    xMeans = np.mean(xMat, axis = 0)                    #行与行操作，求均值
    xVar = np.var(xMat, axis = 0)                        #行与行操作，求方差
    xMat = (xMat - xMeans) / xVar                        #数据减去均值除以方差实现标准化
    numTestPts = 30                                        #30个不同的lambda测试
    wMat = np.zeros((numTestPts, np.shape(xMat)[1]))    #初始回归系数矩阵
    for i in range(numTestPts):                            #改变lambda计算回归系数
        ws = ridgeRegres(xMat, yMat, np.exp(i - 10))    #lambda以e的指数变化，最初是一个非常小的数，
        wMat[i, :] = ws.T                                 #计算回归系数矩阵
    return wMat

In [26]:
 """
 函数说明:交叉验证岭回归
参数说明：
    xMat：数据集特征
    yMat：数据集标签
    numVal： 交叉验证次数
 """
def crossValidation(xArr, yArr, numVal = 10):
    m = len(yArr)                                                                        #统计样本个数                       
    indexList = list(range(m))                                                            #生成索引值列表
    errorMat = np.zeros((numVal,30))                                                    #create error mat 30columns numVal rows
    for i in range(numVal):                                                                #交叉验证numVal次
        trainX = []; trainY = []                                                        #训练集
        testX = []; testY = []                                                            #测试集
        
        random.shuffle(indexList)                                                        #打乱次序
        
        for j in range(m):                                                                #划分数据集:90%训练集，10%测试集
            if j < m * 0.9:
                trainX.append(xArr[indexList[j]])
                trainY.append(yArr[indexList[j]])
            else:
                testX.append(xArr[indexList[j]])
                testY.append(yArr[indexList[j]])
                
        wMat = ridgeTest(trainX, trainY)                                                #获得30个不同lambda下的岭回归系数
        for k in range(30):                                                                #遍历所有的岭回归系数
            matTestX = np.mat(testX); matTrainX = np.mat(trainX)                        #测试集
            meanTrain = np.mean(matTrainX,0)                                            #测试集均值
            varTrain = np.var(matTrainX,0)                                                #测试集方差
            matTestX = (matTestX - meanTrain) / varTrain                                 #测试集标准化
            
            yEst = matTestX * np.mat(wMat[k,:]).T + np.mean(trainY)                        #根据ws预测y值
            errorMat[i, k] = rssError(yEst.T.A, np.array(testY))                            #统计误差
            
    meanErrors = np.mean(errorMat,0)                                                    #计算每次交叉验证的平均误差
    minMean = float(min(meanErrors))                                                    #找到最小误差
    bestWeights = wMat[np.nonzero(meanErrors == minMean)]                                #找到最佳回归系数
    
    xMat = np.mat(xArr); yMat = np.mat(yArr).T
    meanX = np.mean(xMat,0); varX = np.var(xMat,0)
    unReg = bestWeights / varX                                                            #数据经过标准化，因最佳回归系数需要还原
    print('%f%+f*年份%+f*部件数量%+f*是否为全新%+f*原价' % ((-1 * np.sum(np.multiply(meanX,unReg)) + np.mean(yMat)), unReg[0,0], unReg[0,1], unReg[0,2], unReg[0,3]))    
 

In [32]:
if __name__ == '__main__':
    lgX = []
    lgY = []
    useStandRegress()
    setDataCollect(lgX, lgY)
    print("缩减过程中回归系数是如何变化的\n",ridgeTest(lgX, lgY))
    crossValidation(lgX, lgY)

D:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file D:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


62846.112549-31.383276*年份+0.051156*部件数量-10.631107*是否为全新+1.918861*原价
缩减过程中回归系数是如何变化的
 [[-1.09256573e+02  3.02376928e+04  6.16634513e-02  4.37026895e+04]
 [-1.01437283e+02  1.48309142e+04  1.13101868e+00  4.33891367e+04]
 [-9.74026618e+01  7.21909662e+03  2.75589516e+00  4.09392444e+04]
 [-9.52249364e+01  3.80392868e+03  5.83032974e+00  3.49312308e+04]
 [-9.35865345e+01  2.06523622e+03  1.07483665e+01  2.48545383e+04]
 [-9.22167179e+01  1.02230740e+03  1.60348103e+01  1.39168803e+04]
 [-9.12770709e+01  4.42804960e+02  1.96857266e+01  6.33565350e+03]
 [-9.06329675e+01  1.75108476e+02  2.14916551e+01  2.55382781e+03]
 [-8.98460954e+01  6.62714068e+01  2.22068382e+01  9.73785402e+02]
 [-8.81500357e+01  2.46201787e+01  2.23779251e+01  3.63112450e+02]
 [-8.39773054e+01  9.06886028e+00  2.21721096e+01  1.34256413e+02]
 [-7.44255943e+01  3.32003746e+00  2.14694091e+01  4.94848752e+01]
 [-5.67853426e+01  1.20789980e+00  2.00202093e+01  1.82205039e+01]
 [-3.43511318e+01  4.38264788e-01  1.7784977

看运行结果的第一行，可以看到最大的是第4项，第二大的是第2项。

因此，如果只选择一个特征来做预测的话，我们应该选择第4个特征，也就是原始加个。如果可以选择2个特征的话，应该选择第4个和第2个特征。

这种分析方法使得我们可以挖掘大量数据的内在规律。在仅有4个特征时，该方法的效果也许并不明显；但如果有100个以上的特征，该方法就会变得十分有效：它可以指出哪个特征是关键的，而哪些特征是不重要的。